## **Lab 06-3: Softmax Classifier Tensorflow 구현하기**

In [2]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)  # for reproducibility
tfe = tf.contrib.eager

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [1]:
# Sample Dataset

x_data = [
    [1,2,1,1],
    [2,1,3,2],
    [3,1,3,4],
    [4,1,5,5],
    [1,7,5,5],
    [1,2,5,6],
    [1,6,6,6],
    [1,7,7,7],
]

y_data = [
    [0,0,1],
    [0,0,1],
    [0,0,1],
    [0,1,0],
    [0,1,0],
    [0,1,0],
    [1,0,0],
    [1,0,0],
]

In [3]:
# convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32) # list -> numpy
y_data = np.asarray(y_data, dtype=np.float32) # list -> numpy

nb_classes = 3 # num classes

In [11]:
# weight and bias setting
W = tfe.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W,b]
variables

[<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
 array([[ 2.0265348 , -0.19990598,  0.18759498],
        [-1.8624718 ,  1.1830902 , -0.75108325],
        [ 0.7819291 ,  0.19707595,  0.6640797 ],
        [ 1.5643852 , -0.04990807, -0.38255563]], dtype=float32)>,
 <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.4564867 ,  0.53983474, -1.1366715 ], dtype=float32)>]

In [18]:
# tf.nn.softmax computes softmax activations
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X,W) + b)

In [19]:
hypothesis(x_data)

<tf.Tensor: id=140, shape=(8, 3), dtype=float32, numpy=
array([[2.4881808e-02, 9.6873927e-01, 6.3789245e-03],
       [9.8632777e-01, 1.2180600e-02, 1.4916507e-03],
       [9.9994230e-01, 5.2787913e-05, 4.8965262e-06],
       [9.9999952e-01, 3.5200765e-07, 8.7773259e-08],
       [4.1362971e-05, 9.9995792e-01, 7.1108065e-07],
       [9.9881989e-01, 1.1706438e-03, 9.4601755e-06],
       [7.7800662e-03, 9.9221438e-01, 5.5833375e-06],
       [3.3521655e-03, 9.9664682e-01, 9.2721103e-07]], dtype=float32)>

In [20]:
# softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db,dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[1.0000000e+00 5.2308366e-08 1.9130997e-09]], shape=(1, 3), dtype=float32)


In [23]:
# cost function(cross_entropy)
def cost_fn(X,Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.log(logits), axis = 1)
    cost_mean = tf.reduce_mean(cost)
    return cost_mean

print(cost_fn(x_data,y_data))

tf.Tensor(6.9942255, shape=(), dtype=float32)


In [27]:
def grad_fn(X,Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X,Y)
        grad = tape.gradient(loss,variables)
    return grad

print(grad_fn(x_data,y_data))

[<tf.Tensor: id=230, shape=(4, 3), dtype=float32, numpy=
array([[ 1.0009195 , -0.25209376, -0.7488257 ],
       [-0.9869866 ,  1.4851966 , -0.4982102 ],
       [ 0.3810178 ,  0.49261218, -0.87363005],
       [ 0.50757205,  0.3662426 , -0.8738147 ]], dtype=float32)>, <tf.Tensor: id=229, shape=(3,), dtype=float32, numpy=array([ 0.2526431 ,  0.12137035, -0.37401348], dtype=float32)>]


In [33]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs + 1):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if i % 100 == 0:
            print('Loss at epoch %d: %f' %(i, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data

Loss at epoch 0: 0.040280
Loss at epoch 100: 0.039896
Loss at epoch 200: 0.039520
Loss at epoch 300: 0.039151
Loss at epoch 400: 0.038788
Loss at epoch 500: 0.038432
Loss at epoch 600: 0.038082
Loss at epoch 700: 0.037739
Loss at epoch 800: 0.037401
Loss at epoch 900: 0.037070
Loss at epoch 1000: 0.036744
Loss at epoch 1100: 0.036424
Loss at epoch 1200: 0.036109
Loss at epoch 1300: 0.035800
Loss at epoch 1400: 0.035496
Loss at epoch 1500: 0.035197
Loss at epoch 1600: 0.034903
Loss at epoch 1700: 0.034613
Loss at epoch 1800: 0.034329
Loss at epoch 1900: 0.034049
Loss at epoch 2000: 0.033773


In [34]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[1.3606435e-04 1.6270988e-02 9.8359299e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)
